# Exercises

## Setup

In [15]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm

## Exercise 1: Make some art

In the section "The world's simplest model" from "Lesson 2: Regression Refresher" we created the following visualization.

<center>
    <img src="imgs/intercept_only_model_curve.png" style="width:950px;"/>
</center>

This allowed us to see the intercept-only model fits a flat line that is not affected by the values of the predictor. 

You're asked to reproduce this visualization, usin the Bambi model we created in this lesson. Fortunately, you don't need to write a lot of code from scratch. Use the snippet from Lesson 2 and find the places where you need to change parameter names. 

```python
fig, ax = plt.subplots(figsize=FIGSIZE)
ax.scatter(x=data["Length1"], y=data["Weight"], alpha=0.6)
ax.set(xlabel="Length (centimeters)", ylabel="Weight (grams)", title="Fish length vs weight");

for value in idata.posterior["β0"].to_numpy().flatten()[::10]:
    ax.axhline(value, color="C1", alpha=0.2)

β0_mean = idata.posterior["β0"].to_numpy().mean()
ax.axhline(β0_mean, color="C4")

handles = [
    Line2D([], [], label="Observations", lw=0, marker="o", color="C0", alpha=0.6),
    Line2D([], [], label="Posterior draws", lw=2, color="C1"),
    Line2D([], [], label="Posterior mean", lw=2, color="C4"),
]
ax.legend(handles=handles, loc="upper left");
```

## Exercise 2: Deepen your `plot_cap()` knowledge

In this exercise you have to work with the fish data and the model we created in the "Transformations in Bambi" section. More concrete, you need to build the following model

```python
`model = bmb.Model("log(Weight) ~ 1 + log(Length1)", data)`
```
Notice you need to import the data to solve this exercise. Then create and fit the model mentioned above, and use `plot_cap()` with `Length1` as the variable for the horizontal axis.

```python
plot_cap(model, idata, "Length1")
```

Answer the following questions

* What do you see?
* What is the scale of the variables in the horizontal and the vertical axis? Consider they're used with an inline transformation in the model.
    * Why?
* How can you put both variables in the transformed scale?
* And how you put everything on the untransformed scale?

## Exercise 3: Advanced operators

In the section "Transformations in Bambi" we showed that some custom operations needed to be written using a particular syntax. In this exercise you're going to experiment a little with with these operations to see how they work in practice. The goal here is not to arrive to a "correct" answer, but to see what are the different results when we try different formulas and trying to understand what's going on. Use the following simulated data:

In [2]:
rng = np.random.default_rng(1211)
size = 100
df = pd.DataFrame(
    {
        "response": rng.normal(loc=5, scale=0.5, size=size),
        "x": rng.normal(loc=10, scale=2, size=size),
        "y": rng.normal(size=size),
    }
)

Use the following formulas and explain what's happening

* `"response ~ x / 100`"
* `"response ~ x ** 100"`
* `"response ~ x / y"`
* `"response ~ (x + y) ** 2"`

It's fine if you can only describe what you see. At this point you're not expected to be an expert in formula notation. We'll cover more advanced stuff later in the course.

What is the difference between `"response ~ x + y"` and `"response ~ I(x + y)"`?

## Exercise 4: Experiment!

In the "Parameter identifiability" section we simulated some data to grasp what non-identifiability means using a controlled scenario. The code we used is the following:

In [ ]:
rng = np.random.default_rng(1234)
b0, b1 = 0.5, 2.5
x = np.linspace(0, 3, num=50)
noise = rng.normal(scale=0.5, size=50)
y = b0 + b1 * x + noise

And then we created the following PyMC model

In [ ]:
with pm.Model() as weird_model:
    b0 = pm.Normal("b0", sigma=0.5)
    b1 = pm.Normal("b1")
    b2 = pm.Normal("b2")
    mu = b0 + b1 * x + b2 * x
    sigma = pm.HalfNormal("sigma" ,sigma=0.5)
    pm.Normal("y", mu=mu, sigma=sigma, observed=y)

The goal of this exercise is to have you run several experiments to understand how parameter non-identifiablity affects sampling times and the quality of the draws obtained. You are asked to

* Repeat this experiment with different sample sizes.
    * Use $n=500$ and $n=2000$
* Explore the posterior of the slopes
* Change the seeds and comment on the posterior results
* Do you get any warnings?
    * Explore ESS and R-hat
    * Explore the correlation between $b_1$ and $b_2$. 
        * Does it improve with more data?

Now, use the correct model, with a single slope

```python
with pm.Model() as good_model:
    b0 = pm.Normal("b0", sigma=0.5)
    b1 = pm.Normal("b1")
    mu = b0 + b1 * x
    sigma = pm.HalfNormal("sigma" ,sigma=0.5)
    pm.Normal("y", mu=mu, sigma=sigma, observed=y)
```

* Fit it using $n=500$ and $n=2000$, as done previously
* Compare the sampling speed with the previous model
    * What can you conclude about the effects of parameter correlations in sampling speed?
* Can you recover the true slope?
* Do you get any warnings? Again explore ESS and R-hat.
* Also explore the correlation between model parameters. 
    What can you conclude?

Finally, you're asked to reproduce the flawed model in Bambi. To do so, you need to use the formula `"y ~ x + x"`. 

* What happens?
* Why do you think it works that way?

## Exercise 5

* Fit the model using other numerical predictors, compute the $R^2$, and compare to what we got here.

## Exercise 6

Some real world use case